# Reward Model and Evaluation Models

In [ ]:
import torch
print(torch.__version__)

1.13.1+cu117


In [ ]:
# !pip install evaluate

In [ ]:
# !pip install sacremoses sacrebleu

## 4. EASSE Library: complete library for multiple metrics

Ref: https://github.com/feralvam/easse

Features
- Automatic evaluation metrics (e.g. SARI1, BLEU, SAMSA, etc.).
- Commonly used evaluation sets.
- Literature system outputs to compare to.
- Word-level transformation analysis.
- Referenceless Quality Estimation features.
- Straightforward access to commonly used evaluation datasets.
- Comprehensive HTML report for quantitative and qualitative evaluation of a simplification output.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
os.getcwd()

'/home/jovyan/1B-ML2'

In [ ]:
# Delete the failed easse file before git clone
import shutil

# remove old account directory
shutil.rmtree('/home/jovyan/1B-ML2/easse')

In [ ]:
!git clone https://github.com/feralvam/easse.git

Cloning into 'easse'...
remote: Enumerating objects: 1960, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1960 (delta 107), reused 95 (delta 95), pack-reused 1827
Receiving objects: 100% (1960/1960), 33.14 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (1229/1229), done.
Updating files: 100% (203/203), done.


In [ ]:
# %cd easse
# !pip install -e .

In [22]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=["About 95 species are currently accepted.", "The cat perched on the mat."],  
            sys_sents=["About 95 you now get in.", "Cat on mat."], 
            refs_sents=[["About 95 species are currently known.", "The cat sat on the mat."],
                        ["About 95 species are now accepted.", "The cat is on the mat."],  
                        ["95 species are now accepted.", "The cat sat."]])

ModuleNotFoundError: ignored

## Test EASSE on TST and Turk
- turk: https://huggingface.co/datasets/turk

In [ ]:
# !git clone https://github.com/cocoxu/simplification

Cloning into 'simplification'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 211 (delta 23), reused 26 (delta 19), pack-reused 180
Receiving objects: 100% (211/211), 1.96 MiB | 19.44 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
from datasets import list_datasets
datasets_list = list_datasets()

In [ ]:
len(datasets_list)

19012

In [ ]:
import datasets
data = datasets.load_dataset('turk')

Found cached dataset turk (/home/jovyan/.cache/huggingface/datasets/turk/simplification/1.0.0/f58c3ae4f9e0bf4a71d3b800c355b072171a78dee1357c903664aa27353ba3e8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
turk_test = data['test']

In [ ]:
turk_test = turk_test.to_pandas()

In [ ]:
turk_test.head()

,original,simplifications
0,one side of the armed conflicts is composed ma...,[one side of the armed conflicts consist of th...
1,"jeddah is the principal gateway to mecca , isl...","[jeddah is the main gateway to mecca , islam '..."
2,the great dark spot is thought to represent a ...,[the great dark spot is thought to show a hole...
3,"his next work , saturday , follows an especial...",[his next work is saturday which is an eventfu...
4,"the tarantula , the trickster character , spun...",[the tricky tarantula spun a black web and att...


In [ ]:
turk_test['simplifications'][0][0]

'one side of the armed conflicts consist of the sudanese military and the sudanese militia group janjaweed .'

In [ ]:
len(turk_test)

359

In [ ]:
sim_corpus = []
for i in range(len(turk_test)):
    sim_1 = turk_test['simplifications'][i][0]
    sim_corpus.append(sim_1)

In [ ]:
len(sim_corpus)

359

In [ ]:
# sim_corpus

In [ ]:
org_corpus = turk_test['original']

In [ ]:
len(org_corpus)

359

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu
# corpus_bleu(sim_corpus, org_corpus)

In [ ]:
# from torchmetrics import BLEUScore
# metric = BLEUScore()
# metric(org_corpus, sim_corpus)


In [ ]:
# Compute metric using output of pre-trained TST
def load_txt_to_list(path: str) -> list:
    with open(path, 'r', encoding='utf-8') as f: 
        return f.read().splitlines()

In [ ]:
tst_output = load_txt_to_list('tst.output')

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu
# corpus_bleu(tst_output, sim_corpus)

In [ ]:
# from evaluate import load
# sari = load("sari")
# sari_score = sari.compute(sources=[org_corpus[0]], predictions=[tst_output[0]], references=turk_test['simplifications'][0])


In [ ]:
turk_test['simplifications'][0]

array(['one side of the armed conflicts consist of the sudanese military and the sudanese militia group janjaweed .',
       'one side of the armed conflicts is mainly sudanese military and the janjaweed , which recruited from the afro-arab abbala tribes .',
       'one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribes in sudan .',
       'one side of the armed conflicts is made up mostly of the sudanese military and the janjaweed , a sudanese militia group whose recruits mostly come from the afro-arab abbala tribes from the northern rizeigat region in sudan .',
       'the sudanese military and the janjaweed make up one of the armed conflicts , mostly from the afro-arab abbal tribes in sudan .',
       'one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribe

In [ ]:
tst_output[0]

'the side of the armed is composed mainly the and the janjaweed , a militia group recruited mostly from the afro-arab tribes northern .'

In [ ]:
# SARI
from easse.sari import corpus_sari
corpus_sari(orig_sents=org_corpus,  
            sys_sents=tst_output, 
            refs_sents=[sim_corpus])


39.969959427051094

In [ ]:
# BLEU
from easse.bleu import corpus_bleu
corpus_bleu(sys_sents=tst_output, 
            refs_sents=[sim_corpus])

43.07922442785424

In [ ]:
# FKGL
from easse.fkgl import corpus_fkgl
corpus_fkgl(tst_output)

7.417289525319557

In [ ]:
from easse.report import get_all_scores
get_all_scores(
    orig_sents=org_corpus,
    sys_sents=tst_output,
    refs_sents=[sim_corpus])

{'BLEU': 43.08,
 'SARI': 39.97,
 'FKGL': 7.42,
 'Compression ratio': 0.75,
 'Sentence splits': 0.99,
 'Levenshtein similarity': 0.8,
 'Exact copies': 0.14,
 'Additions proportion': 0.06,
 'Deletions proportion': 0.27,
 'Lexical complexity score': 8.15}

TSToutput is generated by on Jupyter UT because it needs anaconda 

In [ ]:
conda create -n gector python=3.7 

In [ ]:
git clone https://github.com/grammarly/gector.git 

In [ ]:
cd gector

In [ ]:
pip uninstall torch

In [ ]:
pip install torch==1.12.0+cu116 torchvision==0.13.0+cu116 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu116 

In [ ]:
!python predict.py --model_path roberta_1_tst.th  --vocab_path data/output_vocabulary --input_file test.8turkers.tok.norm  --output_file tst.output  --normalize

In [ ]:
T

# Test EASSE on TRL-SIMP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.4 MB/s eta 0:00:00


In [ ]:
cd drive/MyDrive/ml

/content/drive/MyDrive/ml


In [ ]:
cd easse

/content/drive/MyDrive/ml/easse


In [23]:
pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/ml/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-qtuzokqw/tseval_7b6ee7e9f5684aaeade90d1d1af61811
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-qtuzokqw/tseval_7b6ee7e9f5684aaeade90d1d1af61811
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e7397b2
  Preparing metadata (setup.py) ... done
  Attempting uninstall: easse
    Found existing installation: easse 0.2.4
    Can't uninstall 'easse'. No files were found to uninstall.
  Running setup.py develop for easse


In [ ]:
pip install trl-0.2.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./trl-0.2.1.tar.gz
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.7 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.2.1-py3-none-any.whl size=38505 sha256=7455531e54e0a7dfb26a3a410ebcf2c7af44fdeb1b44f73183716fe22175a98e
  Stored in directory: /root/.cache/pip/wheels/4c/b3/b3/aecf6309375667c37cbe45398888de6206878091a1151a35e4
Successfully built trl
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3

In [ ]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model,AutoModelForSeq2SeqLMWithValueHead
from trl.core import respond_to_batch, LengthSampler
from transformers import AutoTokenizer, BartForConditionalGeneration, RobertaForSequenceClassification
import torch

In [ ]:
tokenizerclf = AutoTokenizer.from_pretrained("liamcripwell/ctrl44-clf")
tags=["<ident>", "<para>", "<ssplit>", "<dsplit>"]

clfmodel = RobertaForSequenceClassification.from_pretrained("liamcripwell/ctrl44-clf")

In [ ]:
cd drive/MyDrive/ml

In [ ]:
loadedmodel = AutoModelForSeq2SeqLMWithValueHead(BartForConditionalGeneration.from_pretrained('trl-simp-batch15', cache_dir='trl-simp-batch15', local_files_only=True))

Some weights of the model checkpoint at trl-simp-batch15 were not used when initializing BartForConditionalGeneration: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
loadedtoken =AutoTokenizer.from_pretrained('trl-simp-batch15')

In [ ]:
turk_norm=[]
turk_simp=[]
with open('test.8turkers.tok.norm') as norm:
    turk_norm=norm.readlines()
with open('test.8turkers.tok.simp') as simp:
    turk_simp=simp.readlines()
norm.close()
simp.close()

In [ ]:
def testpipelinemodel(text):
    inputs1 = []
    inputs2 = []
    inputs11 = []
    logits = []
    outputs = []
    simpsentences = []
    
    inputs1 = tokenizerclf(text, return_tensors="pt",padding=True)

    with torch.no_grad():
        logits = clfmodel(**inputs1).logits

    for i in range(len(text)):
        predicted_class_id = logits[i].argmax().item()    
        input2 = tags[predicted_class_id] + ' ' + text[i]
        inputs2.append(input2)     
  
    inputs22 = loadedtoken(inputs2, return_tensors="pt",padding='longest')

    output = loadedmodel.generate(**inputs22, num_beams=10, max_length=128)
    for i in range(len(text)):
         simpsentences.append((loadedtoken.decode(output[i])).strip('<pad>'))

    return inputs1, output, simpsentences

In [ ]:
query_tensor, response_tensor, response = testpipelinemodel(turk_norm[:5])

In [ ]:
turk=response
cleanturk=[]
for i in turk:
  txt=[]
  txt=i.replace('/s>','').replace('<s>','').replace('</s','').strip()
  cleanturk.append([txt])

In [24]:
from easse.report import get_all_scores
get_all_scores(
    orig_sents=org_corpus,
    sys_sents=cleanturk,
    refs_sents=[sim_corpus])

ModuleNotFoundError: ignored

# Evaluation on DRESS

In [ ]:
tstoutput=[]

with open('Dress-Ls') as norm:
    tstoutput=norm.readlines()

norm.close()

In [ ]:

cleanoutput=[]
for i in tstoutput:
  txt=[]
  txt=i.strip('\n').strip()
  cleanoutput.append(txt)

In [ ]:
from easse.report import get_all_scores
get_all_scores(
    orig_sents=org_corpus,
    sys_sents=tstoutput,
    refs_sents=[sim_corpus])